# *sparsesurv*

*sparsesurv* [1] is a toolbox for high-dimensional survival analysis. Currently, the package is focused exclusively on knowledge distillation for sparse survival analysis, sometimes also called preconditioning or reference models [2, 3]. In the future, we plan to also extend sparsesurv to other techniques useful for (high-dimensional) survival analysis that are not commonly available in Python.

Below, we will briefly walk you introduce you to some concepts you should understand before using sparsesurv. Once you're ready to move on, you can check out the basic usage of sparsesurv [XXX].

## Survival analysis

sparsesurv operates on survival analysis data. Below, we quote the notation from the supplementary section of our manuscript to ensure we are on the same page in terms of notation and language.

> In particular survival concerns the analysis and modeling of a non-negative random variable $T > 0$, that is used to model the time until an event of interest occurs. In observational survival datasets, we let $T_i$ and $C_i$ denote the event and right-censoring times of patient $i$. In right-censored survival analysis, we observe triplets $(x_i, \delta_i, O_i)$, where $O_i = \text{min}(T_i, C_i)$ and $\delta_i = {1}(T_i \leq C_i)$. Throughout we assume conditionally independent censoring and non-informative censoring. That is, $T \perp\!\!\!\!\perp C \mid X$ and $C$ may not be a function of any of the parameters of $T$ \citep{kalbfleisch2011statistical}. Further, let $\lambda$ denote the hazard function, $\Lambda$ be the cumulative hazard function, and $S(t) = 1 - F(t)$ be the survival function, where $F(t)$ denotes the cumulative distribution function. We let $\tilde T$ be the set of unique, ascending-ordered death times. $R_i$ is the risk set at time $i$, that is, $R(i) = \{j: O_j \geq O_i\}$. $D_i$ denotes the death set at time $i$, $D(i) = \{j: O_j = i \land \delta_i = 1\}$.

For now, *sparsesurv* operats solely on right censored data, although we may consider an extension to other censoring and truncation schemes, if there is interest. We now briefly show an example right-censored survival dataset available in *scikit-survival* [4], another Python package for survival analysis.

In [1]:
from sksurv.datasets import load_flchain
X, y = load_flchain()

In [2]:
X

,age,chapter,creatinine,flc.grp,kappa,lambda,mgus,sample.yr,sex
0,97.0,Circulatory,1.7,10,5.700,4.860,no,1997,F
1,92.0,Neoplasms,0.9,1,0.870,0.683,no,2000,F
2,94.0,Circulatory,1.4,10,4.360,3.850,no,1997,F
3,92.0,Circulatory,1.0,9,2.420,2.220,no,1996,F
4,93.0,Circulatory,1.1,6,1.320,1.690,no,1996,F
...,...,...,...,...,...,...,...,...,...
7869,52.0,NaN,1.0,6,1.210,1.610,no,1995,F
7870,52.0,NaN,0.8,1,0.858,0.581,no,1999,F
7871,54.0,NaN,NaN,8,1.700,1.720,no,2002,F
7872,53.0,NaN,NaN,9,1.710,2.690,no,1995,F


The design matrix $X$ looks the same as it would in other modeling settings, such as regression or classification, and thus does not require any special treatment from a modeling point of view.

In [3]:
y

array([( True,   85.), ( True, 1281.), ( True,   69.), ...,
       (False, 2507.), (False, 4982.), (False, 3995.)],
      dtype=[('death', '?'), ('futime', '<f8')])

The target $y$ looks weird upon first glance however, as it contains two elements for each sample. These correspond exactly to $O_i$ and $\delta_i$ in our notation section above and respectively represent the censoring indicator and the observed time. Right-censored survival data is generally represented in structured array as is shown here, having one element for the censoring indicator and the observed time:

In [4]:
y["death"]

array([ True,  True,  True, ..., False, False, False])

In [5]:
(y["death"]).astype(int)

array([1, 1, 1, ..., 0, 0, 0])

While the number of covariates relative to the number of is quite good here (only 9 variables for 7,874 samples), a very common setting in (cancer) survival anaylsis is one where the number of available covariates is much larger than the number of available samples (i.e., $p >> n$). This is exactly the setting that *sparsesurv* is designed for. *sparsesurv* is based on knowledge distillation [5], which is also referred to as preconditioning [2] or reference models in statistics [3]. Thus, we briefly introduce the idea of knowledge distillation next.

In [6]:
X.shape

(7874, 9)

## Knowledge distillation

The original idea of knowledge distillation was not directly related interpretabiltiy or feature selection. We note however, that the idea of using something akin to knowledge distillation was used and proposed in statistics before knowledge distillation itself, under the name of preconditioning. We will continue to use the name knowledge distillation since it may be more familiar to readers in the machine learning community.

The actual process of knowledge distillation proceeds in two steps:

    1. Fit a teacher model that can approximate the target of interest (very) well
    
    2. Fit a student model on the predictions of the teacher model, hoping that the teacher model acts as a kind of noise filter

We illustrate how this process can be adapted to survival analysis. Please note that this running example will continue to use `scikit-survival`, before we move on to high-dimensional data.

In [7]:
from sksurv.ensemble import RandomSurvivalForest
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sksurv.metrics import concordance_index_censored

X_train = X.iloc[:5000, [0, 3, 4]]
X_test = X.iloc[5000:, [0, 3, 4]]
y_train = y[:5000]
y_test = y[5000:]

teacher_pipe = make_pipeline(StandardScaler(), RandomSurvivalForest())
student_pipe = make_pipeline(StandardScaler(), LinearRegression())
baseline_pipe = make_pipeline(StandardScaler(), CoxPHSurvivalAnalysis())

teacher_pipe.fit(X_train, y_train)
student_pipe.fit(X_train, teacher_pipe.predict(X_train))
baseline_pipe.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('coxphsurvivalanalysis', CoxPHSurvivalAnalysis())])

In [8]:
concordance_index_censored(y_test["death"], y_test["futime"], teacher_pipe.predict(X_test))[0]

0.5677982023489915

In [9]:
concordance_index_censored(y_test["death"], y_test["futime"], student_pipe.predict(X_test))[0]

0.6479734724443875

In [10]:
concordance_index_censored(y_test["death"], y_test["futime"], baseline_pipe.predict(X_test))[0]

0.6444986225266496

Interestingly, in this example, the student performance (as measured by Harrell's concordance) was slightly higher than the baseline, despite the teacher performing quite bad. There is ongoing research in the ML community along these lines. For us, however, all that matters is that knowledge distillation works for survival analysis.

## Minimal example of *sparsesurv*

Lastly, we give a brief example of usage of *sparsesurv*. If you are interested in using *sparsesurv* on your own data, please consult the documentation or the more specific user guides linked above.

In [11]:
import pandas as pd
from sparsesurv.utils import transform_survival
df = pd.read_csv("https://zenodo.org/records/10027434/files/OV_data_preprocessed.csv?download=1")


In [14]:
X = df.iloc[:, 3:].to_numpy()

In [15]:
y = transform_survival(time=df.OS_days.values, event=df.OS.values)

In [16]:
X.shape

(302, 19076)

In [17]:
y[: 5]

array([( True,  304.), ( True,   24.), (False,  576.), (False, 1207.),
       ( True,  676.)], dtype=[('event', '?'), ('time', '<f8')])

In [18]:
from sklearn.decomposition import PCA
from sparsesurv._base import KDSurv
from sparsesurv.cv import KDPHElasticNetCV
from sparsesurv.utils import transform_survival

pipe = KDSurv(
            teacher=make_pipeline(
                StandardScaler(),
                PCA(n_components=16),
                CoxPHSurvivalAnalysis(ties="efron"),
            ),
            student=make_pipeline(
                StandardScaler(),
                KDPHElasticNetCV(
                    tie_correction="efron",
                    l1_ratio=0.9,
                    eps=0.01,
                    n_alphas=100,
                    cv=5,
                    stratify_cv=True,
                    seed=None,
                    shuffle_cv=False,
                    cv_score_method="linear_predictor",
                    n_jobs=1,
                    alpha_type="min",
                ),
            ),
        )

In [19]:
pipe.fit(X, y)

Now, we can easily check how many non-zero coefficients the fitted model has, or get predictions on the training set.

In [25]:
import numpy as np
np.sum(pipe.student[1].coef_ != 0.0)

260

In [26]:
pipe.predict(X)

array([ 2.82819899e-01,  4.66079463e-01,  1.15606183e-01,  5.02952325e-01,
        4.17393777e-02, -4.11573164e-01,  7.60555538e-01,  4.34577104e-01,
        1.14479065e-01,  3.75259090e-02, -1.34671115e-02,  1.61175017e-01,
        1.00036148e-01,  2.40373951e-01, -3.25340021e-01, -8.67875954e-01,
        7.96211926e-01, -1.83116380e-01,  1.07281533e-02, -4.77575349e-02,
       -1.43752928e-01,  4.76747822e-01, -1.28454611e-02, -2.77780752e-01,
        1.18303704e-01,  7.13485684e-01, -9.06133817e-02, -2.28805873e-01,
       -1.85240257e-01, -1.88895164e-01,  1.79877864e-02, -4.40709840e-01,
       -3.33191668e-02, -3.47169497e-01, -5.41839603e-02, -7.00115696e-01,
       -4.83739189e-01,  1.49624486e-01,  2.72062364e-01,  7.45285449e-01,
       -7.67869829e-02, -1.82008113e-01, -6.69662687e-02,  8.93221182e-02,
        2.10881649e-01, -5.40481656e-01, -6.76554743e-02,  4.59392323e-02,
       -1.14925641e-01,  1.08297960e-01, -1.48445361e-01,  3.92129238e-01,
        3.46609042e-04,  

## References

[1] Our manuscript is still under review.

[2] Paul, Debashis, et al. "“Preconditioning” for feature selection and regression in high-dimensional problems." (2008): 1595-1618.

[3] Pavone, Federico, et al. "Using reference models in variable selection." Computational Statistics 38.1 (2023): 349-371.

[4] Pölsterl, Sebastian. "scikit-survival: A Library for Time-to-Event Analysis Built on Top of scikit-learn." The Journal of Machine Learning Research 21.1 (2020): 8747-8752.

[5] Hinton, Geoffrey, Oriol Vinyals, and Jeff Dean. "Distilling the knowledge in a neural network." arXiv preprint arXiv:1503.02531 (2015).